In [1]:
# Using livecoinwatch API
## https://livecoinwatch.github.io/lcw-api-docs/?python#coinslist
## (Install this "requests" library) python -m pip install requests
## !{sys.executable} -m pip install python_dotenv

In [2]:
# read from local .env file
import os
from dotenv import load_dotenv
load_dotenv() 

API_KEY = os.environ['LIVE_COIN_WATCH_API_KEY']


In [3]:
# CONFIG
## SET this to get ATH/50
ATH_PERCENTAGE_THRESHOLD = 2 # <======= 2 tuc la 2% (ATH chia 50)


## API query config
### COIN TO ASK PER REQUEST (API LIMIT TO 100)
SINGLE_QUERY_SIZE = 100
### HOW MANY TIME WE CALL API
QUERY_RANGE = 5
### TOTAL COIN TO QUERY is SINGLE_QUERY_SIZE * QUERY_RANGE

In [4]:
# QUERY
import requests
import json

url = "https://api.livecoinwatch.com/coins/list"


def get_coin_price(limit, offset):
    payload = json.dumps({
      "currency": "USD",
      "sort": "rank",
      "order": "ascending",
      "offset": offset,
      "limit": limit,
      "meta": True
    })
    headers = {
      'content-type': 'application/json',
      'x-api-key': API_KEY
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.text

In [5]:
# PROGRESS BAR
from ipywidgets import IntProgress
from IPython.display import display
import time

max_count = QUERY_RANGE

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

count = 0

# MAIN
import pandas as pd
pd.options.mode.copy_on_write = True
pd.set_option('display.max_rows', 700)


df = pd.DataFrame()
## QUERY and read into dataframe
for index in range(0, QUERY_RANGE - 1):
    # update progress bar
    f.value += 1
    # query
    json_data = get_coin_price(limit=SINGLE_QUERY_SIZE, offset=index*SINGLE_QUERY_SIZE)
    temp_df = pd.read_json(json_data)
    ## Get only price collumns
    temp_df = temp_df[["name", "code", "rate", "allTimeHighUSD"]]
    df = pd.concat([df,temp_df]).reset_index(drop=True)


## Calculate ATH percentage
df["percentage_off_ATH"] = (df["rate"] / df["allTimeHighUSD"]) * 100
df["percentage_off_ATH"] = df["percentage_off_ATH"].round(decimals=2)

## rename
df = df.rename(columns={"rate": "price"})

## Filter by percentage
df = df.loc[df['percentage_off_ATH'] < ATH_PERCENTAGE_THRESHOLD]


print('List of coins with price < ' + str(ATH_PERCENTAGE_THRESHOLD) + '% of ATH')

display(df)

print('Scan Completed!')

IntProgress(value=0, max=4)

List of coins with price < 2% of ATH


,name,code,price,allTimeHighUSD,percentage_off_ATH
30,Internet Computer,ICP,5.159435,494.563642,1.04
32,Filecoin,FIL,4.671353,237.646690,1.97
46,BitTorrent,BTT,0.000001,0.014668,0.01
56,Luna Classic,LUNC,0.000177,118.737617,0.00
64,Mina Protocol,MINA,0.802633,187.815860,0.43
67,Flow,FLOW,0.760809,42.483073,1.79
91,Zcash,ZEC,30.536661,2217.640000,1.38
94,Casper Network,CSPR,0.041768,2.340000,1.78
97,Flare,FLR,0.016301,2.449580,0.67
113,Dash,DASH,32.668202,1642.220000,1.99


Scan Completed!
